In [1]:
from IPython.display import display, Math
import pyvista
from dolfinx import mesh, fem, plot, io, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
from mpi4py import MPI
import ufl
import numpy as np
from dolfinx.io import gmshio
from ufl import SpatialCoordinate
from petsc4py import PETSc
from dolfinx.plot import vtk_mesh



In [2]:


E_bulk = 100*10**8
E_inclusion =150*10**8

nu_inclusion = 0.35
nu_bulk = 0.35



traction =  (0,0)
traction2 = (0,0)
traction3 = (0,0)
traction4 = (0,0)

In [3]:
L = 0.01
N=10
num_inc = 25

msh = mesh.create_rectangle(MPI.COMM_WORLD,points=[(0,0),(num_inc*L,num_inc*L)],n=(num_inc*N,num_inc*N),cell_type=mesh.CellType.quadrilateral)



# def upper(x):
#     return (x[1] >=L/3)  &(x[1] <= 2*L/3)

# def righter(x):
#     return (x[0] >=L/3)  &(x[0] <= 2*L/3)



# def middle(x):
#     return x[1] >= L/2

def mid1(x):
     
    return(np.logical_and.reduce((
        x[1] > L/3 -L/(N),
        x[1] < 2*L/3 +L/(N),
        x[0] > L/3 -L/(N),
        x[0] < 2*L/3 +L/(N)
    )))   


def mid2(x):
    return(np.logical_and.reduce((
        x[1] > L/3 -L/(N),
        x[1] < 2*L/3 +L/(N),
        x[0] > L/3 -L/(N) + L,
        x[0] < 2*L/3 +L/(N) + L
    )))   
def mid3(x):
     
    return(np.logical_and.reduce((
        x[1] > L/3 -L/(N),
        x[1] < 2*L/3 +L/(N),
        x[0] > L/3 -L/(N) + 2*L,
        x[0] < 2*L/3 +L/(N)  + 2*L
    )))   



# def middle(x):
#     return np.logical_or(np.logical_or(
#         mid1(x),
#         mid3(x)
#     ),mid2(x))
        
def middle(x):
    period = L
    # x_mod = x.copy()
    # x_mod[0] = x[0] % period

    # Central region bounds within a cell
    buffer = L / N  # inclusion size control
    low = L / 3 - buffer
    high = 2 * L / 3 + buffer

    return np.logical_and.reduce((
        x[0]%period > low,
        x[0]%period < high,
        x[1]%period     > low,
        x[1]%period    < high
    ))    


# def middle(x):
#     return x[0] >7182312

def b1(x):
    return (np.logical_or.reduce((
        x[1] < L/3,
        x[1] > 2*L/3,
        np.logical_and(0<=x[0],x[0] < L/3) ,
        np.logical_and(L>x[0],x[0] > 2*L/3)
    )))
def b2(x):
     return (np.logical_or.reduce((
        x[1] < L/3,
        x[1] > 2*L/3,
        np.logical_and(L< x[0] ,x[0]< L/3 + L),
        np.logical_and(2*L>x[0] ,x[0]> 2*L/3 + L)
    )))
def b3(x):
     return (np.logical_or.reduce((
        x[1] < L/3,
        x[1] > 2*L/3,
        np.logical_and(2*L <x[0],x[0] < L/3 + 2*L),
        np.logical_and(3*L>=x[0] ,x[0]> 2*L/3 +2* L)
    )))


# def bulk(x):
#    return np.logical_or.reduce((
#         b1(x),
#         b2(x),
#         b3(x)
#     ))
def bulk(x):
    return np.logical_not(middle(x))

middle_cells = mesh.locate_entities(msh, dim=2, marker=middle)

bulk_cells = mesh.locate_entities(msh, dim=2, marker=bulk)
print(bulk_cells, middle_cells)


cell_tags = np.full(msh.topology.index_map(2).size_local, -1, dtype=np.int32)
cell_tags[middle_cells] = 1
cell_tags[bulk_cells] = 2  

msh.topology.create_connectivity(2, 0)

cell_markers = mesh.meshtags(msh, 2, np.concatenate([middle_cells, bulk_cells]),
                 np.concatenate([np.full(len(middle_cells), 1, dtype=np.int32),
                                 np.full(len(bulk_cells), 2, dtype=np.int32)]))


        

[    0     1     2 ... 62497 62498 62499] [   24    31    32 ... 62467 62468 62475]


In [4]:
# domain = mesh.create_unit_square(MPI.COMM_WORLD, nx, ny,cell_type=mesh.CellType.quadrilateral)

# msh, cell_markers, facet_markers = gmshio.read_from_msh("randomInclusions2DCirlce.msh", MPI.COMM_WORLD, gdim=2)
cell_tags = np.full(msh.topology.index_map(2).size_local, -1, dtype=np.int32)

V = fem.functionspace(msh, element = ("CG", 1, (2,)) )

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)


cell_markers.values

array([1, 1, 1, ..., 2, 2, 2], shape=(50000,), dtype=int32)

In [5]:
def clamped_boundary(x):
    
    return  np.isclose(x[1], 0.0)
    # return np.isclose(x[1],0)
def clamped_boundary2(x):
    # if True in (np.isclose(x[1],0.01) & np.isclose(x[0], 0.01)):
    #     print('True')
    return np.isclose(x[1],0.01)
def clamped_boundary3(x):
    return np.isclose(x[0],0) 

def clamped_boundary4(x):
    return np.isclose(x[0], num_inc*0.01)

def clamped_boundary0(x):
    return np.isclose(x[0], 23131231) 

def traction_boundary(x):
    return np.isclose(x[1],  0)

def traction_boundary2(x):
    return np.isclose(x[1],  0.01)

def traction_boundary3(x):
    return np.isclose(x[0], 0)

def traction_boundary4(x):
    return np.isclose(x[0], 0.01)



fdim =msh.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(msh, fdim, clamped_boundary)
boundary_facets2 = mesh.locate_entities_boundary(msh, fdim, clamped_boundary2)
boundary_facets3 = mesh.locate_entities_boundary(msh, fdim, clamped_boundary3)
boundary_facets4 = mesh.locate_entities_boundary(msh, fdim, clamped_boundary4)
boundary_facets0 = mesh.locate_entities_boundary(msh, fdim, clamped_boundary0)

In [6]:
u_D1 = np.array([0.00001,0.00], dtype=default_scalar_type)

# u_D1 = fem.Constant(msh, PETSc.ScalarType(0.0))
bc1 = fem.dirichletbc(u_D1, fem.locate_dofs_topological(V, fdim, boundary_facets3), V)


u_D2 = np.array([-0.00001,-0.00], dtype=default_scalar_type)
bc2 = fem.dirichletbc(u_D2, fem.locate_dofs_topological(V, fdim, boundary_facets4), V)


# u_D3 = np.array([0.0,0.0000000000], dtype=default_scalar_type)
# def f_bc(x):
#     zero = fem.Constant(msh, default_scalar_type((1, 0)))
#     return ufl.dot(x, zero)

# x = SpatialCoordinate(msh)
# f_bc = lambda x: ufl.as_vector(np.stack((x[0],np.zeros_like(x[0]))))


# ubc = f_bc(x)
# def u_b3_exp(x):
#     # print(x)
#     # print(np.shape(np.vstack((x[0], np.zeros_like(x[0])))))
#     # return np.vstack((x[0], np.zeros_like(x[0])),dtype=default_scalar_type)
#     zero = fem.Constant(msh, default_scalar_type((1, 0)))
#     return ufl.dot(x, zero)

# u_b3_exp = ufl.as_vector([x[0],0])
u_b3_func = fem.Function(V)
# # d = f_bc(u)
# u_b3_exp = fem.Expression(ubc
#     ,
#     V.element.interpolation_points(),
    

# u_b3_func.interpolate(lambda x: np.vstack([x[0],np.zeros_like(x[0])], dtype=default_scalar_type),fem.locate_dofs_topological(V, fdim, boundary_facets3))
# # u_b3_func.interpolate(lambda x: np.stack((x[0], 0)))
# print(u_b3_func.x.array.shape)
# bc3 = fem.dirichletbc(u_b3_func, fem.locate_dofs_topological(V, fdim, boundary_facets3), V)

# def f2(x): 
#     return -x[0]

# # u_D4 =  np.array([0,-0.0000000000], dtype=default_scalar_type)
# u_b4 = fem.Function(V)
# u_b4.interpolate(f2)
# bc4 = fem.dirichletbc(u_b4, fem.locate_dofs_topological(V, fdim, boundary_facets4), V)


bc0 = fem.dirichletbc( np.array([0,-0.0000000000], dtype=default_scalar_type), fem.locate_dofs_topological(V, fdim, boundary_facets0), V)

traction_boundary_facets1 = mesh.locate_entities_boundary(msh, fdim, traction_boundary)
traction_boundary_facets2 = mesh.locate_entities_boundary(msh, fdim, traction_boundary2)
traction_boundary_facets3 = mesh.locate_entities_boundary(msh, fdim, traction_boundary3)
traction_boundary_facets4 = mesh.locate_entities_boundary(msh, fdim, traction_boundary4)

facet_markers1 = np.full(len(traction_boundary_facets1), 1, dtype=np.int32)
facet_markers2 = np.full(len(traction_boundary_facets2), 2, dtype=np.int32)
facet_markers3 = np.full(len(traction_boundary_facets3), 3, dtype=np.int32)
facet_markers4 = np.full(len(traction_boundary_facets4), 4, dtype=np.int32)


facet_tags1 = mesh.meshtags(msh, fdim, traction_boundary_facets1,facet_markers1)
facet_tags2 = mesh.meshtags(msh, fdim, traction_boundary_facets2,facet_markers2)
facet_tags3 = mesh.meshtags(msh, fdim, traction_boundary_facets3,facet_markers3)
facet_tags4 = mesh.meshtags(msh, fdim, traction_boundary_facets4,facet_markers4)


combined_facets = np.concatenate([traction_boundary_facets1, traction_boundary_facets2,traction_boundary_facets3, traction_boundary_facets4])
combined_markers = np.concatenate([facet_markers1, facet_markers2, facet_markers3, facet_markers4])

facet_tags = mesh.meshtags(msh, fdim, combined_facets, combined_markers)





In [7]:
Elastic_func_space = fem.functionspace(msh,("DG",0))


E = fem.Function(Elastic_func_space)
nu = fem.Function(Elastic_func_space)


# E.x.array[:] = np.where(cell_markers.values == 1, E_inclusion, E_bulk)

E.x.array[middle_cells] =np.full_like(middle_cells,E_inclusion, dtype=default_scalar_type)
E.x.array[bulk_cells] =np.full_like(bulk_cells,E_bulk, dtype=default_scalar_type)

nu.x.array[middle_cells] = np.full_like(middle_cells,nu_inclusion, dtype=default_scalar_type)
nu.x.array[bulk_cells] = np.full_like(bulk_cells,nu_bulk, dtype=default_scalar_type)


for index,val in enumerate(E.x.array):
    if val == 0:
       E.x.array[index] = E_bulk
       nu.x.array[index] = nu_bulk
if 0 in E.x.array:
    assert False, "E is zero"

In [8]:
ds = ufl.Measure("ds", domain=msh, subdomain_data=facet_tags )

# plane strain
lam = E*nu/((1+nu)*(1-2*nu))
mu = E/(2*(1+nu))

# plane stress
# lambda_ = 2*lam*mu/(lam + 2*mu)
lambda_ = E*nu/(1-nu**2)



def epsilon(u):
    return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
    

def sigma(u):
    return lam * ufl.tr(epsilon(u)) * ufl.Identity(len(u)) + 2 * mu * epsilon(u)

    

In [9]:

T = fem.Constant(msh, default_scalar_type(traction))

T2 = fem.Constant(msh, default_scalar_type(traction2))

T3= fem.Constant(msh, default_scalar_type(traction3))
T4 = fem.Constant(msh, default_scalar_type(traction4))




f = fem.Constant(msh, default_scalar_type((0, 0)))

a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
L = ufl.dot(f, v) * ufl.dx + ufl.dot(T, v) *ds(1)  + ufl.dot(T2, v) *ds(2) + ufl.dot(T3, v) *ds(3) + ufl.dot(T4, v) *ds(4)
# L = ufl.dot(f, v) * ufl.dx  

problem = LinearProblem(a, L, bcs=[bc1,bc2], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()



In [10]:

topology, cell_types, geometry = plot.vtk_mesh(V)
g = pyvista.UnstructuredGrid(topology, cell_types, geometry)

g.point_data["u"] = uh.x.array.reshape((geometry.shape[0], 2))

if g.point_data['u'].shape[1] == 2:
    g.point_data['u'] = np.hstack((g.point_data['u'], np.zeros((geometry.shape[0], 1))))
g.point_data['u']
warped = g.warp_by_vector("u", factor=5)
g.cell_data["E"] = E.x.array
g.set_active_scalars("E")


(<FieldAssociation.CELL: 1>,
 pyvista_ndarray([1.e+10, 1.e+10, 1.e+10, ..., 1.e+10, 1.e+10, 1.e+10],
                 shape=(62500,)))

In [11]:
pyvista.start_xvfb()
pyvista.set_jupyter_backend('html')

plotter = pyvista.Plotter()
plotter.add_mesh(g,show_edges=False)
# plotter.add_mesh(warped, show_edges=False,cmap  ='jet')
plotter.view_xy()
plotter.show()


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [12]:
s_indicies = [0,0]

s_indicies_all = {'[0, 0]':'11','[0, 1]':'12','[1, 1]':'22'}


In [13]:
s = sigma(uh)[s_indicies[0],s_indicies[1]]
eps = epsilon(uh)[s_indicies[0],s_indicies[1]]

In [14]:
V_interp = fem.functionspace(msh, ("CG",1))

stresses = fem.Function(V_interp)
strains = fem.Function(V_interp)


stress_expr = fem.Expression(s, V_interp.element.interpolation_points())
strains_expr = fem.Expression(eps, V_interp.element.interpolation_points())


stresses.interpolate(stress_expr)
strains.interpolate(strains_expr)

topology, cell_types, geometry = plot.vtk_mesh(V_interp)




In [15]:


grid_s = pyvista.UnstructuredGrid(topology, cell_types, geometry)

legend_name = "S" + s_indicies_all[str(s_indicies)]

grid_s.point_data[legend_name] = stresses.x.petsc_vec.array


print(geometry.shape,uh.x.array.shape,stresses.x.array.shape)

grid_s.point_data["u"] = uh.x.array.reshape((geometry.shape[0], 2))

if grid_s.point_data['u'].shape[1] == 2:
    grid_s.point_data['u'] = np.hstack((grid_s.point_data['u'], np.zeros((geometry.shape[0], 1))))
grid_s.point_data['u']

warped = grid_s.warp_by_vector("u", factor=100)




# print(geometry)

grid_s.set_active_scalars(legend_name)

p = pyvista.Plotter()
# warped.set_active_scalars(legend_name)

# p.add_mesh(grid_s,show_edges=False,cmap = "jet")

p.add_mesh(warped,show_edges=False,cmap = "jet")


p.show_axes()
p.view_xy()
p.show()



(63001, 3) (126002,) (63001,)


MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen
D3D12: Removing Device.


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [16]:
geometry.shape, stresses.x.array.shape,strains.x.array.shape



((63001, 3), (63001,), (63001,))

In [17]:
indices = [[0,0],[1,1],[0,1]]

stresses = fem.Function(V_interp)
strains = fem.Function(V_interp)

S_arr  = []
Eps_arr = []

for s_index in indices:
 
    s_indices = s_index

    s = sigma(uh)[s_indices[0],s_indices[1]]
    # print(s)
    eps = epsilon(uh)[s_indices[0],s_indices[1]]
    integral_stress = fem.assemble_scalar(fem.form(s * ufl.dx))/(0.01)**2
    intergal_strain = fem.assemble_scalar(fem.form(eps * ufl.dx))/(0.01)**2
    print('value of integral:',integral_stress, intergal_strain)

    stress_expr = fem.Expression(s, V_interp.element.interpolation_points())
    strains_expr = fem.Expression(eps, V_interp.element.interpolation_points())


    stresses.interpolate(stress_expr)
    strains.interpolate(strains_expr)

    
    S_arr.append(np.mean(stresses.x.petsc_vec.array.copy()))
    if s_indices[0] == 0 and s_indices[1] == 1:
        Eps_arr.append(np.mean(2*strains.x.petsc_vec.array.copy()))
    else:
        Eps_arr.append(np.mean(strains.x.petsc_vec.array.copy()))

S = np.vstack((S_arr[0],S_arr[1],S_arr[2]))
Eps = np.vstack((Eps_arr[0],Eps_arr[1],Eps_arr[2]))
# topology, cell_types, geometry = plot.vtk_mesh(V_interp)
S,Eps

# .0001

value of integral: -656817064.0954846 -0.05000000000000039
value of integral: -92000415.82483838 0.02141250769646042
value of integral: 2.8327735712707636e-05 3.648537616254271e-15


(array([[-1.05048687e+06],
        [-1.48382172e+05],
        [ 4.19704648e-08]]),
 array([[-7.99491249e-05],
        [ 3.41561794e-05],
        [ 1.07068738e-17]]))